In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
import os
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import warnings
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from selenium.webdriver.firefox.options import Options
import numpy as np


In [2]:
def check_substring(inputStr:str):
    DataType = ""
    inputStr = inputStr.lower()

    # can you help do this HAHAHAHAH plessss
    if "criminal data" in inputStr or "criminal record" in inputStr:
        DataType += "Criminal Records, " 
    
    if "personal data" in inputStr or "cvs" in inputStr or "ssn" in inputStr or "personal" in inputStr:
        DataType += "Personal Data, "
    
    if "network data" in inputStr or "network map" in inputStr or "remote services" in inputStr or "local services" in inputStr:
        DataType += "Network Data, "

    if "data of employees" in inputStr or "employee data" in inputStr or "employees data" in inputStr or "data of employee" in inputStr:
        DataType += "Employee Data, "

    if "data of customers" in inputStr or "customer data" in inputStr or "customers data" in inputStr or "data of customer" in inputStr:
        DataType += "Customer Data, "

    if "sensitive" in inputStr:
        DataType += "Sensitive (general), "

    if "database backup" in inputStr or "database backups" in inputStr or "database" in inputStr or "databases" in inputStr:
        DataType += "Database Data, "

    if len(DataType)<1:
        DataType += "Unknown"

    return DataType

In [3]:
df = pd.DataFrame()

torexe = os.popen(r'/home/kali/Downloads/tor-browser/Browser/TorBrowser/Tor/')
profile = FirefoxProfile(r'/home/kali/Downloads/tor-browser/Browser/TorBrowser/Data/Browser/profile.default')
profile.set_preference('network.proxy.type', 1)
profile.set_preference('network.proxy.socks', '127.0.0.1')
profile.set_preference('network.proxy.socks_port', 9150)
profile.set_preference("network.proxy.socks_remote_dns", True)
profile.update_preferences()

#checking if tor is working
driver = webdriver.Firefox(firefox_profile= profile, executable_path=r'/home/kali/Downloads/geckodriver')
website = driver.get("http://check.torproject.org")#checking
options = Options()
options.headless = True
list_desc = []
actual_desc = []

for j in range(1,38):
    time.sleep(3) #sleep for a while before creating the new one
    pageNo = j
    print(pageNo)
    #set a new driver
    driver = webdriver.Firefox(options=options, firefox_profile= profile, executable_path=r'/home/kali/Downloads/geckodriver')
    #get the url 
    url = "http://alphvmmm27o3abo3r2mlmjrpdmzle3rykajqc5xsj7j7ejksbpsa36ad.onion/?page="+str(pageNo)
    website = driver.get(url)

    loaded = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='content']")))
    time.sleep(3) #in case loading does not work 
    each_block = WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//div[@class='content']/div//div[@class='posts']//div[@class='post']//div[@class='post-body']")))
    length = len(each_block)
    for i in each_block:
        company_name = i.find_element(By.XPATH, ".//div[@class='post-header']").text #you need the '.' #self-notes to be cleaned later!
        date = i.find_element(By.XPATH, ".//div[@class='post-subheader small'][1]").text
        try:
            desc = i.find_element(By.XPATH, ".//div[@class='post-description']//span").text
        except:
            desc = "NULL"
        if len(desc)<1:
            desc = "NULL"
        if desc != "NULL":
            DataType = check_substring(str(desc))        
        df2 = pd.DataFrame({'CompanyName': company_name,"date": date, "description":desc, "Data Type":DataType}, index=[0])
        df = pd.concat([df,df2], axis = 0)
    
    
    
    for j in range(0, length):
        time.sleep(3)
        driver = webdriver.Firefox(options=options, firefox_profile= profile, executable_path=r'/home/kali/Downloads/geckodriver')
        website = driver.get(url)
        loaded = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='content']")))
        try: 
            print("Button Number : ", j)
            buttons =  [my_elem for my_elem in WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.XPATH, "//div[@class='content']/div//div[@class='posts']//div[@class='post']//div[@class='post-footer-right']/a")))] 
            buttons[j].click()
            loaded = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//div[@class='post-message']")))
            DataTypeDesc = driver.find_element(By.XPATH, "//div[@class='post-message']").text
            list_desc.append(check_substring(str(DataTypeDesc)))
            actual_desc.append(str(DataTypeDesc))
            driver.back();
        except Exception as e:
            if len(str(DataTypeDesc))<1:
                list_desc.append("Unknown")
                actual_desc.append("Unknown")
            print(e)


    print("Page ", pageNo, " Done")
    

        



driver.quit()


/bin/sh: 1: /home/kali/Downloads/tor-browser/Browser/TorBrowser/Tor/: Permission denied


1


KeyboardInterrupt: 

In [ ]:
df['Data Type 2'] = list_desc
df['description 2'] = actual_desc

# Using clearnet to scrap the company information

In [ ]:
companies_list = df['CompanyName'].to_list()
IndustryType = []
Websites = []
Address = []

df.to_csv('ALPHV_DATA.csv')

In [ ]:
import requests

def location_to_country(location):
    base_url = "https://nominatim.openstreetmap.org/search"
    
    params = {
        "addressdetails": 1,
        "q": location,
        "format": "jsonv2",
        "limit": 1
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if data:
        country = data[0].get("address", {}).get("country", "")
        return country

    return np.nan

In [ ]:
# Using LinkedIn

companyWebsite_list = []
industry_list = []
headquarter_list = []
country_list = []

wait = WebDriverWait(driver, 10)

for company in companies_list:
    print("start: ", end =" ")
    time.sleep(5)
    profile = FirefoxProfile(r'/home/kali/Downloads/tor-browser/Browser/TorBrowser/Data/Browser/profile.default')
    profile.update_preferences()
    options = Options()
    options.headless = True
    driver = webdriver.Firefox(options=options, firefox_profile= profile, executable_path=r'/home/kali/Downloads/geckodriver')
    try:
        driver.get('https://www.google.com')
        search = driver.find_element(By.NAME, 'q')
        search.send_keys(company + ' linkedin')
        search.send_keys(Keys.RETURN)
        loaded = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "main")))
        linkedin_page = driver.find_element_by_tag_name('h3') # clicking the first search 
        print(linkedin_page.text, end =": ")
        time.sleep(10) # to limit rate of requests to linkedin
        linkedin_page.click()

        if "linkedin" in driver.current_url:

            loaded = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "main-content")))
            try:
                sign_in_button_element = driver.find_element_by_xpath("//button[@class='sign-in-modal__outlet-btn cursor-pointer btn-md btn-primary']")
                sign_in_button_element.click()
                dismiss_button_element = driver.find_element_by_xpath("//button[@class='modal__dismiss btn-tertiary h-[40px] w-[40px] p-0 rounded-full indent-0 sign-in-modal__dismiss absolute right-0 cursor-pointer m-[20px]']")
                dismiss_button_element.click()
            except:
                pass

            CLASS = ".//dd[@class='font-sans text-md text-color-text break-words overflow-hidden']"

            try:
                website_element = driver.find_element_by_css_selector('[data-test-id="about-us__website"]')
                website_text_element = website_element.find_element(By.XPATH, CLASS)
                website_text = website_text_element.text
            except Exception as e:
                website_text = np.nan
                print(e)
            companyWebsite_list.append(website_text)

            try:
                industry_element = driver.find_element_by_css_selector('[data-test-id="about-us__industry"]')
                industry_text_element = industry_element.find_element(By.XPATH, CLASS)
                industry_text = industry_text_element.text
            except Exception as e:
                industry_text = np.nan
                print(e)
            industry_list.append(industry_text)
            
            try:
                headquarters_element = driver.find_element_by_css_selector('[data-test-id="about-us__headquarters"]')
                headquarters_text_element = headquarters_element.find_element(By.XPATH, CLASS)
                headquarters_text = headquarters_text_element.text

                country = location_to_country(headquarters_text)
            except Exception as e:
                headquarters_text = np.nan
                country = np.nan
                print(e)
            headquarter_list.append(headquarters_text)
            country_list.append(country)

        else:
            companyWebsite_list.append(np.nan)
            industry_list.append(np.nan)
            headquarter_list.append(np.nan)
            country_list.append(np.nan)

    except Exception as e:
        companyWebsite_list.append(np.nan)
        industry_list.append(np.nan)
        headquarter_list.append(np.nan)
        country_list.append(np.nan)
        print(e)
    finally:
        print("Finished")
        driver.quit()

    

In [ ]:
df["Company Website"] = companyWebsite_list
df["Industry"] = industry_list
df["Location"] = headquarter_list
df["Country"] = country_list

In [ ]:
df.to_csv('ALPHV_LINKEDIN_DATA.csv')

In [ ]:
df